In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=62d7ca2101e8edb4de5bf4bcb98bd7a2f1512df726a30d061306c486ba2c14ba
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
pip install findspark

In [ ]:
import pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('capstone').getOrCreate()

In [ ]:
products = spark.read.csv("products.csv",header=True)

In [ ]:
orders=spark.read.csv("orders.csv",header=True)

In [ ]:
customers=spark.read.csv("customers.csv",header=True)

In [ ]:
products.show(5)

+---+---------+--------------------+-------------+------------------+------+--------------------+--------------------+
| ID| Category|          Created At|          Ean|             Price|Rating|               Title|              Vendor|
+---+---------+--------------------+-------------+------------------+------+--------------------+--------------------+
|  1|    Gizmo|2017-07-19T19:44:...|1018947080336|29.463261130679875|   4.6| Rustic Paper Wallet|Swaniawski, Caspe...|
|  2|Doohickey|2019-04-11T08:49:...|7663515285824| 70.07989613071763|   0.0|  Small Marble Shoes|Balistreri-Ankunding|
|  3|Doohickey|2018-09-08T22:03:...|4966277046676|35.388744881539054|   4.0|Synergistic Grani...|Murray, Watsica a...|
|  4|Doohickey|2018-03-06T02:53:...|4134502155718| 73.99178100854834|   3.0|Enormous Aluminum...|Regan Bradtke and...|
|  5|   Gadget|2016-10-03T01:47:...|5499736705597|  82.7450976850356|   4.0|Enormous Marble W...|Price, Schultz an...|
+---+---------+--------------------+------------

In [ ]:
orders.show(5)

+---+--------------------+-----------------+----------+--------+------------------+----+------------------+-------+
| ID|          Created At|         Discount|Product ID|Quantity|          Subtotal| Tax|             Total|User ID|
+---+--------------------+-----------------+----------+--------+------------------+----+------------------+-------+
|  1|2019-02-11T21:40:...|             NULL|        14|       2|37.648145389078365|2.07|39.718145389078366|      1|
|  2|2018-05-15T08:04:...|             NULL|       123|       3|110.93145648834248| 6.1| 117.0376564084763|      1|
|  3|2019-12-06T22:22:...|6.416679208849759|       105|       2|52.723521442619514| 2.9| 55.62208681964182|      1|
|  4|2019-08-22T16:30:...|             NULL|        94|       6|109.21864156655383|6.01| 115.2207354961295|      1|
|  5|2018-10-10T03:34:...|             NULL|       132|       5|127.88197029833711|7.03|134.94192935296473|      1|
+---+--------------------+-----------------+----------+--------+--------

In [ ]:
customers.show(5)

+---+-------------------+--------------------+----------+-----------+--------------------+--------------------+-----------------+------------+--------------------+---------+-----+-----+
| ID|               Name|             Address|Birth Date|       City|          Created At|               Email|         Latitude|   Longitude|            Password|   Source|State|  Zip|
+---+-------------------+--------------------+----------+-----------+--------------------+--------------------+-----------------+------------+--------------------+---------+-----+-----+
|  1|       Hudson Borer|9611-9809 West Ro...|1986-12-12| Wood River|2017-10-07T01:34:...|borer-hudson@yaho...|40.71314890000001| -98.5259864|ccca881f-3e4b-4e5...|  Twitter|   NE|68883|
|  2|Domenica Williamson|      101 4th Street|1967-06-10|  Searsboro|2018-04-09T12:10:...|williamson-domeni...|       41.5813224| -92.6991321|eafc45bf-cf8e-4c9...|Affiliate|   IA|50242|
|  3|        Lina Heaney|29494 Anderson Drive|1961-12-18|  Sandstone|2

In [ ]:
customer_orders = customers.join(orders, customers["ID"] == orders["User ID"])

combined = customer_orders.join(products, customer_orders["Product ID"] == products["ID"])

combined.show()

+---+------------+--------------------+----------+----------+--------------------+--------------------+-----------------+-----------+--------------------+--------+-----+-----+---+--------------------+------------------+----------+--------+------------------+-----+------------------+-------+---+---------+--------------------+-------------+------------------+------+--------------------+--------------------+
| ID|        Name|             Address|Birth Date|      City|          Created At|               Email|         Latitude|  Longitude|            Password|  Source|State|  Zip| ID|          Created At|          Discount|Product ID|Quantity|          Subtotal|  Tax|             Total|User ID| ID| Category|          Created At|          Ean|             Price|Rating|               Title|              Vendor|
+---+------------+--------------------+----------+----------+--------------------+--------------------+-----------------+-----------+--------------------+--------+-----+-----+---+---

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum

# Initialize Spark
spark = SparkSession.builder.appName('SalesAnalysis').getOrCreate()



# Group by city and calculate total sales, total quantity, etc.
city_sales = combined.groupBy("City").agg(
    sum("Total").alias("TotalSales"),
    sum("Quantity").alias("TotalQuantity")
)

# Group by state and calculate total sales, total quantity, etc.
state_sales = combined.groupBy("State").agg(
    sum("Total").alias("TotalSales"),
    sum("Quantity").alias("TotalQuantity")
)

# Group by source and calculate total sales, total quantity, etc.
source_sales = combined.groupBy("Source").agg(
    sum("Total").alias("TotalSales"),
    sum("Quantity").alias("TotalQuantity")
)

# Display or save the results as needed
city_sales.show()

+-------------------+------------------+-------------+
|               City|        TotalSales|TotalQuantity|
+-------------------+------------------+-------------+
|           DeRuyter|129.74883608352175|         10.0|
|          Fairbanks| 6389.123328767112|        253.0|
|            Shannon|1146.4334629763896|         46.0|
|          West York|  898.225484719706|         28.0|
|        Springfield|2011.3425790883814|        131.0|
|      Bowling Green| 922.9827230198426|         43.0|
|              Ryder|  668.923028231959|         42.0|
|            Pukwana|1509.5891761453502|         39.0|
|         Harrisburg| 351.7789519569911|         12.0|
|             Auburn| 324.3136506099869|         24.0|
|        River Falls| 1455.166101687321|         76.0|
|           Brinkley| 686.5963271362704|         31.0|
|           Harrison| 541.3065105285232|         16.0|
|        Kings Point|2068.5231848599797|         62.0|
|         Wolfe City| 712.3500245452483|         17.0|
|         

In [ ]:
state_sales.show()

+-----+------------------+-------------+
|State|        TotalSales|TotalQuantity|
+-----+------------------+-------------+
|   AZ| 6407.199291732651|        237.0|
|   SC|29210.479408564457|       1163.0|
|   LA|20740.508087059607|        875.0|
|   MN| 69064.01586770012|       3056.0|
|   NJ| 6768.687499559493|        314.0|
|   OR| 32415.70034927819|       1487.0|
|   VA|29211.249189820894|       1219.0|
|   KY| 41082.72106482593|       1987.0|
|   WY|21035.722877885597|       1009.0|
|   NH| 2775.888499371694|         73.0|
|   MI|52237.678569649455|       2191.0|
|   NV| 3346.731494837816|        174.0|
|   WI| 59732.90421633871|       2300.0|
|   ID|17428.889547384915|        679.0|
|   CA| 52559.37866225034|       2261.0|
|   NE| 45512.11043736533|       1953.3|
|   CT| 2206.771524765544|         88.0|
|   MT|  69436.1363004223|       3310.0|
|   NC| 48834.20587334776|       2241.0|
|   VT|7279.1304098682795|        355.0|
+-----+------------------+-------------+
only showing top

In [ ]:
source_sales.show()

+---------+-----------------+-------------+
|   Source|       TotalSales|TotalQuantity|
+---------+-----------------+-------------+
|  Organic|319637.0497447002|      14079.0|
|  Twitter|319448.7269814239|      13981.3|
|   Google|325183.5607881129|      14324.0|
|Affiliate|297605.2749875903|      12892.0|
| Facebook|333453.5126582293|      14261.0|
+---------+-----------------+-------------+



In [ ]:
source_sales.write.csv('source_sales.csv')

AnalysisException: ignored

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum

# Initialize Spark
spark = SparkSession.builder.appName('SalesAnalysis').getOrCreate()

# Load the customers, orders, and products DataFrames (Assuming you've already loaded the data)

# Join customers, orders, and products DataFrames
# combined_df = customers.join(orders, "ID", "inner").join(products, "ID", "inner")

# Group by city and calculate total sales, total quantity, etc.
city_sales = combined.groupBy("City").agg(
    sum("Total").alias("TotalSales"),
    sum("Quantity").alias("TotalQuantity")
)

# Group by state and calculate total sales, total quantity, etc.
state_sales = combined.groupBy("State").agg(
    sum("Total").alias("TotalSales"),
    sum("Quantity").alias("TotalQuantity")
)

# Group by source and calculate total sales, total quantity, etc.
source_sales = combined.groupBy("Source").agg(
    sum("Total").alias("TotalSales"),
    sum("Quantity").alias("TotalQuantity")
)

# Display or save the results as needed
city_sales.show()

+-------------------+------------------+-------------+
|               City|        TotalSales|TotalQuantity|
+-------------------+------------------+-------------+
|           DeRuyter|129.74883608352175|         10.0|
|          Fairbanks| 6389.123328767112|        253.0|
|            Shannon|1146.4334629763896|         46.0|
|          West York|  898.225484719706|         28.0|
|        Springfield|2011.3425790883814|        131.0|
|      Bowling Green| 922.9827230198426|         43.0|
|              Ryder|  668.923028231959|         42.0|
|            Pukwana|1509.5891761453502|         39.0|
|         Harrisburg| 351.7789519569911|         12.0|
|             Auburn| 324.3136506099869|         24.0|
|        River Falls| 1455.166101687321|         76.0|
|           Brinkley| 686.5963271362704|         31.0|
|           Harrison| 541.3065105285232|         16.0|
|        Kings Point|2068.5231848599797|         62.0|
|         Wolfe City| 712.3500245452483|         17.0|
|         